### Imports

In [16]:
import pandas as pd
from collections import defaultdict
import requests
from typing import List
import os
import io
import numpy as np
import re

pd.set_option('display.max_columns', 500)

## Installation and Downloads

blah blah blah, this will have what sources I am using and how I got them.

## Getting our Balance Data

In [6]:
def check_columns(dataframes: pd.DataFrame) -> List[pd.DataFrame]:
    col_hash = defaultdict(list)
    for df in dfs:
        col_len = len(df.columns)
        col_hash[col_len].append(df)
    while len(col_hash) > 1:
        max_col = max(col_hash.keys())
        min_col = min(col_hash.keys())
        cols = set(col_hash[max_col][0].columns)
        cur_df = col_hash[min_col].pop()
        cur_cols = cur_df.columns
        diff = list(cols.difference(cur_cols))
        cur_df[diff] = np.nan
        col_hash[max_col].append(cur_df)
        if len(col_hash[min_col]) == 0:
            del col_hash[min_col]
    return col_hash[max_col]

In [47]:
dfs = []
threshold = 0.8
pattern = r'\([^()]*\)|\b(from|at|of)\b'
for year in range(2016, 2025):
    firsthalf_link = f"https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_{year}_Jan_Jun.csv"
    sechalf_link = f"https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_{year}_Jul_Dec.csv"
    
    r1 = requests.get(firsthalf_link)
    r2 = requests.get(sechalf_link)
    try:
        data1 = r1.content.decode('utf8')
        data2 = r2.content.decode('utf8')
        d1 = pd.read_csv(io.StringIO(data1), low_memory=False)
        d2 = pd.read_csv(io.StringIO(data2), low_memory=False)
        df = pd.concat([d1, d2], axis=0)
    
        cols = [col for col in df.columns if "Imputed" not in col and "Adjusted" in col]
        columns = list(df.columns[:4]) + cols + ["Region"]
        midw = df[(df['Region'] == "MIDW") & (df['Balancing Authority'] == "MISO")][columns]
        dfs.append(midw)
    except Exception as e:
        print(e)
dfs = check_columns(dfs)
master_df = pd.concat(dfs, axis=0, ignore_index=True)
master_df.columns = ["_".join(re.sub(pattern, '', col).lower().split()) for col in master_df.columns]
master_df["local_time_end_hour"] = pd.to_datetime(master_df["local_time_end_hour"])
master_df =  master_df \
    .sort_values("local_time_end_hour",  ignore_index=True) \
    .dropna(axis=1, thresh=int(len(master_df) * (1-0.7))) \
    .dropna(axis=0, thresh=7) \
    .bfill(axis=0)
master_df.to_csv("../../data/balance_sheet.csv", index=False)

In [49]:
pd.read_csv("../../data/balance_sheet.csv")

,balancing_authority,data_date,hour_number,local_time_end_hour,demand,net_generation,total_interchange,net_generation_coal,net_generation_natural_gas,net_generation_nuclear,net_generation_hydropower_and_pumped_storage,net_generation_solar,net_generation_wind,net_generation_other_fuel_sources,region
0,MISO,01/01/2016,1,2016-01-01 01:00:00,"67,929","65,109","-3,713","39,962","20,101","11,610",746,0,"3,865","1,122",MIDW
1,MISO,01/01/2016,2,2016-01-01 02:00:00,"66,212","63,395","-3,697","39,962","20,101","11,610",746,0,"3,865","1,122",MIDW
2,MISO,01/01/2016,3,2016-01-01 03:00:00,"64,903","61,707","-4,063","39,962","20,101","11,610",746,0,"3,865","1,122",MIDW
3,MISO,01/01/2016,4,2016-01-01 04:00:00,"64,091","60,736","-4,198","39,962","20,101","11,610",746,0,"3,865","1,122",MIDW
4,MISO,01/01/2016,5,2016-01-01 05:00:00,"63,985","60,692","-4,001","39,962","20,101","11,610",746,0,"3,865","1,122",MIDW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76387,MISO,09/17/2024,20,2024-09-17 20:00:00,"92,071","91,551","-2,309","27,832","38,414","11,432","2,432",86,"11,046",309,MIDW
76388,MISO,09/17/2024,21,2024-09-17 21:00:00,"88,200","87,955","-2,025","26,881","35,064","11,438","1,012",79,"13,213",269,MIDW
76389,MISO,09/17/2024,22,2024-09-17 22:00:00,"82,729","81,743","-2,725","24,019","31,176","11,438",408,79,"14,359",265,MIDW
76390,MISO,09/17/2024,23,2024-09-17 23:00:00,"78,145","77,534","-2,545","21,432","28,938","11,445",384,79,"14,972",284,MIDW
